In [1]:
import base_experiment

ROOT_DIR, DATA_PATH, LOG_DIR, DEVICE = base_experiment.setup_experiment(
    seed=20, 
    data_path="dataset/pokec", 
    log_dir="experiments/fair_ac/logs/pokec_n", 
    device=1
)

# import these after, as we need to set correct path in setup_experiment
import torch
import numpy as np
from models.ac import FairAC, Trainer
from dataset import PokecN

Using directories:
root_dir: /home/fact21/fact_refactor
data_dir: /home/fact21/fact_refactor/dataset/pokec
log_dir: /home/fact21/fact_refactor/experiments/logs/pokec_z
device: cuda:1


In [2]:
# Load in the dataset
dataset = PokecN(
    nodes_path=DATA_PATH / "region_job_2.csv",
    edges_path=DATA_PATH / "region_job_2_relationship.txt",
    embedding_path=DATA_PATH / "pokec_n_embedding10.npy",
    feat_drop_rate=0.3,
    device=DEVICE
)

print(f"Loaded dataset with {dataset.graph.num_nodes()} nodes and {dataset.graph.num_edges()} edges")
print(f"Using feat_drop_rate: {dataset.feat_drop_rate}")

/home/fact21/fact_refactor/src/dataset.py:153: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(


Loaded dataset with 67796 nodes and 1303712 edges
Using feat_drop_rate: 0.3


In [3]:
# Create FairAC model
fair_ac = FairAC(
    feature_dim=dataset.features.shape[1],
    transformed_feature_dim=128,
    emb_dim=dataset.embeddings.shape[1],
    attn_vec_dim=128,
    attn_num_heads=1,
    dropout=0.5,
    num_sensitive_classes=1,
)

print(f"Created FairAC model with {1} sensitive class")

Created FairAC model with 1 sensitive class


In [4]:
# Create FairAC trainer
trainer = Trainer(
    ac_model=fair_ac,
    lambda1=1.0,
    lambda2=1.0,
    dataset=dataset,
    device=DEVICE,
    gnn_kind="GCN",
    gnn_hidden_dim=128,
    gnn_lr=1e-3,
    gnn_weight_decay=1e-5,
    gnn_args={"dropout": 0.5},
    log_dir=LOG_DIR,
    min_acc=0.65,
    min_roc=0.69,
)

print(f"Created trainer with {'GCN'} model, using LOG_DIR: {LOG_DIR}")

Created trainer with GCN model, using LOG_DIR: /home/fact21/fact_refactor/experiments/logs/pokec_z


In [5]:
# Run pre-training
trainer.pretrain(epochs=200)
print("Finished pretraining")

  0%|          | 0/200 [00:00<?, ?it/s]

Finished pretraining


In [ ]:
# Main training loop, with GNN validation
trainer.train(val_start_epoch=800, val_epoch_interval=200, epochs=2800)

  0%|          | 0/2800 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]